In [1]:
from functools import partial
import math

import numpy as np
import tensorflow as tf

In [2]:
from ocddetection import data, models

In [3]:
table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(list(data.LABEL2IDX.keys()), list(data.LABEL2IDX.values())),
    0
)

In [4]:
df_train, df_val, _ = data.split(
    data.files('../../data/OpportunityUCIDataset/dataset/'),
    validation=[(1, 2)],
    test=[(2, 3), (2, 4), (3, 3), (3, 4)]
)

In [5]:
val = data \
    .to_centralised(df_val) \
    .map(partial(data.preprocess, sensors=data.SENSORS, label=data.MID_LEVEL, table=table)) \
    .filter(data.filter_nan) \
    .window(64, shift=64 // 2).flat_map(partial(data.windows, window_size=64)) \
    .batch(16)

In [6]:
model = models.build(
    32,
    len(data.LABEL2IDX),
    .2
)

In [7]:
fwd_state = model.initialize_hidden_state(16)
bwd_state = model.initialize_hidden_state(16)

In [ ]:
for X_val, y_val in val:
    y_hat, fwd_state, bwd_state = model(X_val, [fwd_state, bwd_state], training=True)